# PKM-RE: Eksplorasi Data H3N2 Hemagglutinin

## Informasi Sumber Data

| Atribut | Detail |
|---------|--------|
| **Database** | NCBI Protein Database |
| **URL** | https://www.ncbi.nlm.nih.gov/protein |
| **Query** | Influenza A virus H3N2 hemagglutinin human |
| **Tanggal Download** | 18 Januari 2026 |
| **Jumlah Sekuens** | 1500 |
| **Format** | FASTA (Protein) |
| **Panjang Sekuens** | 500-600 asam amino |

### Referensi Database
- NCBI Influenza Virus Resource: https://www.ncbi.nlm.nih.gov/genomes/FLU/
- Entrez Programming Utilities: https://www.ncbi.nlm.nih.gov/books/NBK25501/

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from Bio import SeqIO, Entrez
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

print('Libraries loaded successfully!')
print(f'Analysis date: {datetime.now().strftime("%Y-%m-%d %H:%M")}')

## 1. Load dan Parse Data FASTA

In [ ]:
# Load FASTA file
fasta_file = '../data/raw/h3n2_ha_sequences.fasta'

records = []
for record in SeqIO.parse(fasta_file, 'fasta'):
    records.append({
        'accession': record.id,
        'description': record.description,
        'sequence': str(record.seq),
        'length': len(record.seq)
    })

df = pd.DataFrame(records)
print(f'Total sequences loaded: {len(df)}')
df.head()

## 2. Ekstrak Metadata dari Description

In [ ]:
def extract_year(desc):
    """Extract year from description"""
    # Pattern: A/Location/Number/YEAR
    match = re.search(r'/([12][09]\d{2})\)', desc)
    if match:
        return int(match.group(1))
    # Fallback: any 4-digit year
    match = re.search(r'(20[0-2]\d|19\d{2})', desc)
    if match:
        return int(match.group(1))
    return None

def extract_location(desc):
    """Extract location from strain name"""
    match = re.search(r'A/([^/]+)/', desc)
    if match:
        return match.group(1)
    return 'Unknown'

def extract_strain(desc):
    """Extract full strain name"""
    match = re.search(r'\[(Influenza A virus \([^\]]+\))\]', desc)
    if match:
        return match.group(1)
    return desc

# Apply extraction
df['year'] = df['description'].apply(extract_year)
df['location'] = df['description'].apply(extract_location)
df['strain'] = df['description'].apply(extract_strain)

# Create NCBI link for each sequence
df['ncbi_link'] = df['accession'].apply(lambda x: f'https://www.ncbi.nlm.nih.gov/protein/{x}')

print(f'Years range: {df["year"].min()} - {df["year"].max()}')
print(f'Sequences with year info: {df["year"].notna().sum()}')
df[['accession', 'year', 'location', 'length', 'ncbi_link']].head(10)

## 3. Filter dan Sort by Year (Latest First)

In [ ]:
# Filter sequences with valid year
df_valid = df[df['year'].notna()].copy()
df_valid['year'] = df_valid['year'].astype(int)

# Sort by year descending (latest first)
df_valid = df_valid.sort_values('year', ascending=False).reset_index(drop=True)

print(f'Sequences with valid year: {len(df_valid)}')
print(f'\nYear distribution:')
print(df_valid['year'].value_counts().sort_index(ascending=False).head(15))

In [ ]:
# Visualize year distribution
fig, ax = plt.subplots(figsize=(12, 5))
year_counts = df_valid['year'].value_counts().sort_index()
year_counts.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_xlabel('Year')
ax.set_ylabel('Number of Sequences')
ax.set_title('H3N2 HA Sequences by Collection Year')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../results/year_distribution.png', dpi=150)
plt.show()

## 4. Sequence Length Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram
axes[0].hist(df_valid['length'], bins=30, color='coral', edgecolor='black')
axes[0].set_xlabel('Sequence Length (amino acids)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of HA Sequence Lengths')
axes[0].axvline(df_valid['length'].median(), color='red', linestyle='--', label=f'Median: {df_valid["length"].median()}')
axes[0].legend()

# Boxplot by year (recent years)
recent = df_valid[df_valid['year'] >= 2018]
recent.boxplot(column='length', by='year', ax=axes[1])
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Sequence Length')
axes[1].set_title('Sequence Length by Year (2018+)')
plt.suptitle('')

plt.tight_layout()
plt.savefig('../results/length_analysis.png', dpi=150)
plt.show()

print(f'Length statistics:')
print(df_valid['length'].describe())

## 5. Geographic Distribution

In [ ]:
# Top locations
top_locations = df_valid['location'].value_counts().head(20)

fig, ax = plt.subplots(figsize=(10, 6))
top_locations.plot(kind='barh', ax=ax, color='teal')
ax.set_xlabel('Number of Sequences')
ax.set_ylabel('Location')
ax.set_title('Top 20 Locations for H3N2 HA Sequences')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('../results/location_distribution.png', dpi=150)
plt.show()

## 6. Quality Control & Filtering

In [ ]:
# Filter criteria for ML
MIN_LENGTH = 550
MAX_LENGTH = 580
MIN_YEAR = 2015

df_filtered = df_valid[
    (df_valid['length'] >= MIN_LENGTH) &
    (df_valid['length'] <= MAX_LENGTH) &
    (df_valid['year'] >= MIN_YEAR)
].copy()

# Remove sequences with ambiguous amino acids
df_filtered = df_filtered[~df_filtered['sequence'].str.contains('[XBZ]', regex=True)]

print(f'After filtering:')
print(f'  - Original: {len(df_valid)}')
print(f'  - Filtered: {len(df_filtered)}')
print(f'  - Year range: {df_filtered["year"].min()} - {df_filtered["year"].max()}')
print(f'  - Length range: {df_filtered["length"].min()} - {df_filtered["length"].max()}')

## 7. Save Processed Data with Metadata

In [ ]:
# Save to CSV with full metadata
output_csv = '../data/processed/h3n2_ha_processed.csv'
df_filtered.to_csv(output_csv, index=False)
print(f'Saved {len(df_filtered)} sequences to {output_csv}')

# Save metadata summary
metadata = {
    'source_database': 'NCBI Protein Database',
    'source_url': 'https://www.ncbi.nlm.nih.gov/protein',
    'query': 'Influenza A virus H3N2 hemagglutinin human 500:600[Sequence Length]',
    'download_date': '2026-01-18',
    'total_downloaded': len(df),
    'after_filtering': len(df_filtered),
    'year_range': f'{df_filtered["year"].min()}-{df_filtered["year"].max()}',
    'length_range': f'{df_filtered["length"].min()}-{df_filtered["length"].max()}',
    'filter_criteria': {
        'min_length': MIN_LENGTH,
        'max_length': MAX_LENGTH,
        'min_year': MIN_YEAR,
        'exclude_ambiguous': True
    }
}

import json
with open('../data/processed/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print('Metadata saved to metadata.json')

## 8. Data Provenance Table

In [ ]:
# Create provenance table for top 20 sequences
provenance = df_filtered[['accession', 'strain', 'year', 'location', 'length', 'ncbi_link']].head(20)
provenance.columns = ['Accession', 'Strain Name', 'Year', 'Location', 'Length', 'NCBI Link']

print('\n=== DATA PROVENANCE (Top 20 Latest Sequences) ===')
print('Each sequence can be verified at the NCBI link provided\n')
provenance

In [ ]:
# Save provenance to HTML for documentation
html_table = provenance.to_html(index=False, render_links=True, escape=False)
html_table = html_table.replace('&lt;', '<').replace('&gt;', '>')

with open('../results/data_provenance.html', 'w') as f:
    f.write(f'''
<!DOCTYPE html>
<html>
<head>
    <title>H3N2 Data Provenance</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        table {{ border-collapse: collapse; width: 100%; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        th {{ background-color: #4CAF50; color: white; }}
        tr:nth-child(even) {{ background-color: #f2f2f2; }}
        a {{ color: #0066cc; }}
    </style>
</head>
<body>
    <h1>H3N2 HA Sequence Data Provenance</h1>
    <p><strong>Source:</strong> NCBI Protein Database</p>
    <p><strong>Download Date:</strong> 18 January 2026</p>
    <p><strong>Total Sequences:</strong> {len(df_filtered)}</p>
    {html_table}
</body>
</html>
''')
print('Provenance table saved to results/data_provenance.html')

## Summary

Data telah diproses dan siap untuk tahap selanjutnya:
1. **Feature Extraction** - Ekstraksi fitur fisikokimia
2. **Model Training** - Training XGBoost untuk prediksi antigenic drift